In [2]:
# find average burnup and enrichment
import sqlite3 as lite

conn = lite.connect('1yr.sqlite')
conn.row_factory = lite.Row
cur = conn.cursor()
average_burnup = cur.execute('SELECT avg(discharge_burnup) FROM discharge').fetchone()[0]
average_enrichment = cur.execute('SELECT avg(initial_enrichment) FROM discharge').fetchone()[0]
print('AVERAGE BURNUP : %f MWD/MTHM' %average_burnup)
print('AVERAGE ENRICHMENT: %f %% U235' %average_enrichment)

AVERAGE BURNUP : 36169.381303 MWD/MTHM
AVERAGE ENRICHMENT: 3.392045 % U235


In [19]:
# find an assembly that most represents this datapoint
# the minimization equation is scaled to equally weigh enrichment and burnup
min_diff = cur.execute('SELECT assembly_id, min(abs(discharge_burnup - %f) + abs(initial_enrichment - %f)*10000) '
                       'FROM discharge' %(average_burnup, average_enrichment)).fetchone()[0]
print(min_diff)

324192


In [32]:
# print assembly data
chosen_assem = cur.execute('SELECT * FROM discharge WHERE assembly_id = %i' %min_diff).fetchall()
burnup = chosen_assem[0]['discharge_burnup']
enrichment = chosen_assem[0]['initial_enrichment']
total_mass = chosen_assem[0]['initial_uranium_kg']
print('ENRICHMENT: %f %% U235' %enrichment)
print('BURNUP: %f MWD/MTHM' %burnup)
print('\n')

print('ERROR:')
enr_error = 100 * (average_enrichment - enrichment) / average_enrichment
print('ENRICHMENT: %f %%' %enr_error)
bur_error = 100 * (average_burnup - burnup) / average_burnup
print('BURNUP: %f %%' %bur_error)

print('\n')

pu_comp = 0
u_comp = 0
for row in chosen_assem:
    if 'pu' in row['isotope']:
        pu_comp += float(row['total_mass_g']) * 0.1 / total_mass
    if 'u' == row['isotope'][0]:
        u_comp += float(row['total_mass_g']) * 0.1 / total_mass
    print('%s : %f %%' %(row['isotope'], float(row['total_mass_g']) * 0.1 / total_mass ))

print('\nTOTAL U COMP IN UNF:  %f' %u_comp)
print('TOTAL PU COMP IN UNF:  %f' %pu_comp)


ENRICHMENT: 3.394000 % U235
BURNUP: 36172.000000 MWD/MTHM


ERROR:
ENRICHMENT: -0.057637 %
BURNUP: -0.007240 %


th-230 : 0.000000 %
th-229 : 0.000000 %
u-238 : 93.478329 %
u-236 : 0.431826 %
zr-93 : 0.084079 %
y-90 : 0.000014 %
zr-95 : 0.000107 %
u-233 : 0.000000 %
u-232 : 0.000000 %
th-232 : 0.000000 %
u-235 : 1.013261 %
u-234 : 0.016992 %
pu-239 : 0.731450 %
pu-240 : 0.255995 %
pu-241 : 0.169509 %
pu-242 : 0.057000 %
ru-106 : 0.008052 %
sb-124 : 0.000000 %
se-79 : 0.000502 %
sb-125 : 0.000946 %
ru-103 : 0.000006 %
ra-226 : 0.000000 %
pu-244 : 0.000004 %
sn-126 : 0.002360 %
sm-151 : 0.001558 %
tc-99 : 0.084213 %
sr-90 : 0.053386 %
am-241 : 0.015907 %
am-242m : 0.000122 %
c-14 : 0.000010 %
am-243 : 0.014665 %
cm-242 : 0.000450 %
cf-251 : 0.000000 %
cm-244 : 0.005341 %
cm-243 : 0.000054 %
cf-249 : 0.000000 %
ce-144 : 0.011615 %
cm-245 : 0.000394 %
cm-246 : 0.000031 %
cm-247 : 0.000000 %
cs-134 : 0.010143 %
cm-248 : 0.000000 %
cs-137 : 0.127860 %
cs-135 : 0.055162 %
eu-154 : 0.003002 %


In [35]:
# output this recipe to cyclus xml block
with open('avg_recipe.xml', 'w') as f:
    head = "<recipe>\n\t<name>used_uox_recipe</name>\n\t<basis>mass</basis>\n"
    for row in chosen_assem:
        head += '\t\t<nuclide> <id>%s</id> <comp>%f</comp> </nuclide>\n' %(row['isotope'].capitalize(), float(row['total_mass_g']) * 0.1 / total_mass)
    head += '</recipe>'
    f.write(head)

# this recipe block will be used in the calculations.

In [36]:
!cat avg_recipe.xml

<recipe>
	<name>used_uox_recipe</name>
	<basis>mass</basis>
		<nuclide> <id>Th-230</id> <comp>0.000000</comp> </nuclide>
		<nuclide> <id>Th-229</id> <comp>0.000000</comp> </nuclide>
		<nuclide> <id>U-238</id> <comp>93.478329</comp> </nuclide>
		<nuclide> <id>U-236</id> <comp>0.431826</comp> </nuclide>
		<nuclide> <id>Zr-93</id> <comp>0.084079</comp> </nuclide>
		<nuclide> <id>Y-90</id> <comp>0.000014</comp> </nuclide>
		<nuclide> <id>Zr-95</id> <comp>0.000107</comp> </nuclide>
		<nuclide> <id>U-233</id> <comp>0.000000</comp> </nuclide>
		<nuclide> <id>U-232</id> <comp>0.000000</comp> </nuclide>
		<nuclide> <id>Th-232</id> <comp>0.000000</comp> </nuclide>
		<nuclide> <id>U-235</id> <comp>1.013261</comp> </nuclide>
		<nuclide> <id>U-234</id> <comp>0.016992</comp> </nuclide>
		<nuclide> <id>Pu-239</id> <comp>0.731450</comp> </nuclide>
		<nuclide> <id>Pu-240</id> <comp>0.255995</comp> </nuclide>
		<nuclide> <id>Pu-241</id> <comp>0.169509</comp> </nuclide>
		<nuclide> <id>Pu-242</id> <comp>